# Zero Waveform Generator
Generate a DAC waveform of all zeros with configurable sample rate and duration, plus optional ADC readout file.

In [ ]:
from pathlib import Path
import math

# -----------------------------
# User-configurable parameters
# -----------------------------
SAMPLE_RATE_KSPS = 80.0  # Default sample rate
TOTAL_TIME_MS = 10.0  # Default total time
SPI_CLOCK_FREQ_MHZ = 20.0
OUTPUT_BASENAME = "zero_waveform_80ksps_10ms"

DO_ADC_READOUT = False
ADC_SAMPLE_RATE_KSPS = 62.5
ADC_EXTRA_TIME_MS = 0.5

# Fixed waveform shape
CHANNELS_PER_BOARD = 8
NUM_BOARDS = 1

# -----------------------------
# Helpers
# -----------------------------
MAX_DELAY = 2**25 - 1

def calculate_sample_delay(sample_rate_ksps, spi_clock_freq_mhz):
    sample_rate_hz = sample_rate_ksps * 1000.0
    spi_clock_freq_hz = spi_clock_freq_mhz * 1e6
    cycles_per_sample = spi_clock_freq_hz / sample_rate_hz
    delay = int(round(cycles_per_sample))
    return max(1, min(MAX_DELAY, delay))

def write_adc_readout_file(filename, durations_cycles, adc_sample_rate_ksps, extra_time_ms, spi_clock_freq_mhz):
    extra_cycles = int(round(extra_time_ms * 1e-3 * spi_clock_freq_mhz * 1e6))
    true_extra_time_ms = extra_cycles / (spi_clock_freq_mhz * 1e6) * 1e3
    durations_ms = [cycles / (spi_clock_freq_mhz * 1e6) * 1e3 for cycles in durations_cycles]
    with open(filename, "w") as f:
        f.write("# ADC Readout Command File\n")
        f.write(f"# Extra sample time: {true_extra_time_ms:.6g} ms\n")
        f.write(f"# SPI clock frequency: {spi_clock_freq_mhz:.6g} MHz\n")
        f.write(f"# ADC sample rate: {adc_sample_rate_ksps:.6g} ksps\n")
        f.write(f"# DAC durations (ms): {', '.join(f'{d:.6g}' for d in durations_ms)}\n")
        for dur_cycles in durations_cycles:
            total_cycles = dur_cycles + extra_cycles
            adc_delay_value = calculate_sample_delay(adc_sample_rate_ksps, spi_clock_freq_mhz)
            total_samples = max(1, total_cycles // adc_delay_value)
            f.write("NT 1\n")
            repeat_count = total_samples - 1
            f.write(f"D {adc_delay_value} {repeat_count}\n")

def write_zero_waveform(output_path, total_samples, delay_cycles):
    zero_line = " 0" * CHANNELS_PER_BOARD
    with open(output_path, "w") as f:
        f.write("# Zeroed DAC Waveform File\n")
        f.write(f"# SPI clock frequency: {SPI_CLOCK_FREQ_MHZ:.6g} MHz\n")
        f.write(f"# Sample rate: {SAMPLE_RATE_KSPS:.6g} ksps\n")
        f.write(f"# Number of samples: {total_samples}\n")
        f.write(f"# Channels: {CHANNELS_PER_BOARD}\n")
        f.write("# Format: T <count> <ch0-ch7> (trigger) / D <delay> <ch0-ch7> (delay)\n")
        f.write(f"T 1{zero_line}\n")
        for _ in range(max(0, total_samples - 1)):
            f.write(f"D {delay_cycles}{zero_line}\n")

# -----------------------------
# Main generation
# -----------------------------
sample_rate_hz = SAMPLE_RATE_KSPS * 1000.0
total_time_s = TOTAL_TIME_MS * 1e-3
total_samples = max(1, int(round(total_time_s * sample_rate_hz)))
delay_cycles = calculate_sample_delay(SAMPLE_RATE_KSPS, SPI_CLOCK_FREQ_MHZ)

output_dir = Path(".")
output_dir.mkdir(parents=True, exist_ok=True)
wfm_path = output_dir / f"{OUTPUT_BASENAME}.wfm"
write_zero_waveform(wfm_path, total_samples, delay_cycles)
print(f"Wrote DAC waveform: {wfm_path}")

if DO_ADC_READOUT:
    total_duration_cycles = (total_samples - 1) * delay_cycles
    rdout_path = output_dir / f"{OUTPUT_BASENAME}.rdout"
    write_adc_readout_file(
        rdout_path,
        [total_duration_cycles],
        ADC_SAMPLE_RATE_KSPS,
        ADC_EXTRA_TIME_MS,
        SPI_CLOCK_FREQ_MHZ,
    )
    print(f"Wrote ADC readout: {rdout_path}")
else:
    rdout_path = output_dir / f"{OUTPUT_BASENAME}.rdout"
    with open(rdout_path, "w") as f:
        f.write("# ADC Readout Command File (No readout)\n")
        f.write("# Number of triggers: 1\n")
        f.write("NT 1\n")
    print(f"Wrote ADC readout: {rdout_path}")
